In [1]:
import sympy as sp
import cupy  as cp
import numpy as np

x, y = sp.symbols('x y', real = True)
expr = sp.sin(x*y) + sp.exp(x*1j + y)
expr = sp.simplify(expr)
expr

exp(1.0*I*x + y) + sin(x*y)

In [2]:
import time # zmierzymy prędkość numpy vs cupy
N = 25_000_000


# --- NumPy ---
f_np = sp.lambdify((x, y), expr, modules='numpy')

start = time.perf_counter()

X_np = np.random.uniform(0, 100, N).astype(np.float64)
Y_np = np.random.uniform(0, 100, N).astype(np.float64)

test1 = f_np(X_np,Y_np)

end = time.perf_counter()
print(f"NumPy czas: {end - start:.6f} s")


# --- CuPy ---
f_cp = sp.lambdify((x, y), expr, modules={'sin': cp.sin, 'exp': cp.exp})

start = time.perf_counter()

X_cp = cp.random.uniform(0, 100, N, dtype=cp.float64)
Y_cp = cp.random.uniform(0, 100, N, dtype=cp.float64)

test2 = f_cp(X_cp, Y_cp)

cp.cuda.Stream.null.synchronize() # synchronizacja GPU, aby czas był poprawny
end = time.perf_counter()
print(f"CuPy  czas: {end - start:.6f} s")

NumPy czas: 1.977818 s
CuPy  czas: 0.769787 s


In [ ]:
import time
N = 1_000_000

t = np.random.uniform(0, 100, N).astype(np.float64)

# -----
start = time.perf_counter()

q = ((t >= 3) & (t <= 7)) * np.sin(t + np.exp(t))

end = time.perf_counter()
print(f"{end - start:.6f} s")

# -----
start = time.perf_counter()

q = np.where((t >= 3) & (t <= 7), 1.0, 0.0) * np.sin(t + np.exp(t))

end = time.perf_counter()
print(f"{end - start:.6f} s")

# -----
start = time.perf_counter()

q = (np.heaviside(t-3, 1) - np.heaviside(t-7, 1)) * np.sin(t + np.exp(t))

end = time.perf_counter()
print(f"{end - start:.6f} s")

0.073627 s
0.069626 s
0.073508 s
